In [3]:
import os
%pwd

'f:\\work env\\End-to-End-MLOps-with-MLflow\\research'

In [4]:
os.chdir('../')

In [32]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
  root_dir: Path
  data_path: Path
  schema: dict

In [33]:
from MLOpsProject.constants import *
from MLOpsProject.utils.common import read_yaml, create_directories

In [34]:
# creating folders from the config file
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            schema=self.schema,
        )

        return data_transformation_config

In [49]:
import os
from MLOpsProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

In [58]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.data = pd.read_csv(self.config.data_path, sep=";")
    
    # Adding functionalities: Scaler, Normalizer, etc.
    def categoriacal_columns(self):
        categorical_columns = []
        for col in data.columns:
            if data[col].dtype.name == "object":
                categorical_columns.append(col)
        return categorical_columns

    def handling_missing_values(self):
        for col in self.data.columns:
            if self.data[col].dtype == "object":
                self.data[col].fillna(self.data[col].mode()[0], inplace=True)  # Filling with mode for categorical columns
            else:
                self.data[col].fillna(self.data[col].mean(), inplace=True)  # Filling with mean for numerical columns

    def handle_imbalanced_data(self):
        target = self.config.schema.TARGET_COLUMN.name
        X = self.data.drop(target, axis=1)
        y = self.data[target]

        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)

        self.data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.DataFrame(y_resampled, columns=[target])], axis=1)

    
    def handle_duplicates(self):
        data.drop_duplicates(inplace=True)        

    def handling_outliers(self):
        pass
        # TODO: Add outlier handling code later

    def binning(self):
        categoriacal_columns = self.categoriacal_columns()
        for i in categoriacal_columns:
            data[i] = pd.cut(data[i], bins=5, labels=False)

    def encoding(self):
        categoriacal_columns = self.categoriacal_columns()
        label=LabelEncoder()
        for i in categoriacal_columns:
            data[i]=label.fit_transform(data[i])

    def feature_scaling(self):
        scaler = StandardScaler()
        scaled_data = scaler.fit_transform(data)
        self.data = scaled_data


    def train_test_split(self):
        train, test = train_test_split(data, test_size=0.2, random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        logger.info("splitted data into train and test set")
        logger.info(f"train shape: {train.shape}, test shape: {test.shape}")

        

In [60]:
try: 
    config_manager = ConfigurationManager()
    data_transformation_config = config_manager.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.handling_missing_values()
    data_transformation.handle_duplicates()
    data_transformation.encoding()
    data_transformation.feature_scaling()
    # data_transformation.handle_imbalanced_data()
    data_transformation.train_test_split()
except Exception as e:
    logger.exception(e)
    raise e

[2023-09-28 15:03:55,330: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-28 15:03:55,332: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-28 15:03:55,334: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-28 15:03:55,335: INFO: common: directory is created at artifacts]
[2023-09-28 15:03:55,336: INFO: common: directory is created at artifacts/data_transformation]
[2023-09-28 15:03:55,621: INFO: 563559978: splitted data into train and test set]
[2023-09-28 15:03:55,621: INFO: 563559978: train shape: (23147, 17), test shape: (5787, 17)]
